In [98]:
import librosa
import pandas as pd
import numpy as np
import os
import pathlib
from pathlib import Path
import pickle
from tinytag import TinyTag
from pydub import AudioSegment
import ntpath
from pydub.utils import make_chunks
from numpy  import array
import re
from itertools import tee, islice, chain
from math import atan2,degrees

In [99]:
df = pd.read_pickle("D:/learning/DataScience/data/prj/featureExtraction/picklle_file/split1_feature_extraction_mar_28_1.pkl")

In [100]:
df.head()

,filename,artist,duration,albumartist,bitrate,comment,filesize,genre,title,track,...,tonal_centroid_angle,tempogram_angle,onset_detect_angle,onset_strength_angle,onset_strength_multi_angle,beat_angle,tempo_angle,harmonic_elements_angle,percussive_elements_angle,delta_angle
1,Bebe Rexha - Drinking About You (The Chainsmok...,Bebe Rexha,217.034914,The Chainsmokers Bebe Rexha,320.0,,8712257,Electronic,Drinking About You (The Chainsmokers Remix),None,...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xed\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
2,Bulleya - DJ LIJO & VIKAS J REMIX.mp3,,240.068006,None,320.0,,9608031,None,Bulleya - DJ LIJO & VIKAS J REMIX,None,...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...","b""\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x15\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
3,Chaiyya Chaiyya - DJ LIJO's REMIX.mp3,,248.139043,None,320.0,,9948637,None,Chaiyya Chaiyya - DJ LIJO's REMIX,None,...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\x1d\x02\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
4,Coca Cola - Songs.pk - 128Kbps.mp3,"Tony Kakkar, Neha Kakkar, Young Desi",179.633623,None,128.0,None,3069797,Bollywood Music,Coca Cola - Songs.pk,2,...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'
5,Coca Cola - Songs.pk - 320Kbps.mp3,"Tony Kakkar, Neha Kakkar, Young Desi",179.535321,None,320.0,None,7377494,Bollywood Music,Coca Cola - Songs.pk,2,...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,"b""\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...",b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04\x95\xad\x01\x00\x00\x00\x00\x00\x00...,b'\x80\x04]\x94.'


In [101]:
def calculate_correlation_and_correlationCoeff(x1,x2):
    r = 0
    coeff = 0
    if(x1.shape[0] > x2.shape[0]):
        result = np.zeros(x1.shape)
        result[:x2.shape[0]] = x2
#         case1
        coeff = np.corrcoef(x1,result)
    elif(x1.shape[0] < x2.shape[0]):
        result = np.zeros(x2.shape)
        result[:x1.shape[0]] = x1
#        case2
        coeff = np.corrcoef(result,x2)
    else:
        coeff = np.corrcoef(x1,x2)
    coeff = abs(coeff[0][1])
    return coeff

In [102]:
def getCorrelationCoeffArray(index,count_row):
    row_count_var = 1
    corr_arr = np.array([])
    sorted_arr_rec = []

    while(row_count_var <= count_row ):
        pickle_string = df['audio_waveform'][index]
        audio_waveform_split_input_var = pickle.loads(pickle_string)
        if(row_count_var != index):
            pickle_string = df['audio_waveform'][row_count_var]
            audio_waveform_split_row_count_var = pickle.loads(pickle_string)
            r,coeff = calculate_correlation_and_correlationCoeff(audio_waveform_split_input_var,audio_waveform_split_row_count_var)
            if(row_count_var == 1):
                corr_arr = coeff
            else:
                corr_arr = np.append(corr_arr,coeff)
        row_count_var = row_count_var+1
    corr_arr[::-1].sort()
    for i in corr_arr:
        string = str(i)+','+str(df['filename'][i])
        sorted_arr_rec.append(coeff)
    return sorted_arr_rec
        

In [103]:
def printTopThreeRecommendations(sorted_arr_rec):
    first = sorted_arr_rec[0]
    first_arr = re.split(",",first)
    print(first_arr[1])
    second = sorted_arr_rec[1]
    second_arr = re.split(",",second)
    print(second_arr[1])
    third = sorted_arr_rec[2]
    third_arr = re.split(",",third)
    print(third_arr[1])
    

In [104]:
def getTopThreeRecommendations(index,count_row):

    index_arr = [[0,0]]*6
    row_count_var = 1
    corr_arr = np.array([])
    sorted_arr_rec = []
    pickle_string = df['audio_waveform_split'][index]
    x1 = np.array(pickle.loads(pickle_string))
    r = 0
    while(row_count_var <= count_row ):

        if(row_count_var != index):
            pickle_string = df['audio_waveform_split'][row_count_var]
            x2 = np.array(pickle.loads(pickle_string))
            coeff = calculate_correlation_and_correlationCoeff(x1,x2)
            corr_arr = np.append(corr_arr,coeff)
            corr_list = [coeff,row_count_var]
            index_arr[r] = corr_list
        row_count_var = row_count_var+1
        r += 1
    #     print(row_count_var)
    corr_arr[::-1].sort()

    for za,i in enumerate(corr_arr):
        for a in (index_arr):
            if(i == a[0]):
                string = str(i)+','+str(df['filename'][a[1]])
                sorted_arr_rec.append(string)
    printTopThreeRecommendations(sorted_arr_rec)

In [105]:
count_row = df.shape[0]  # gives number of row count

In [106]:
Recocommendations_For_Index = 5 
getTopThreeRecommendations(Recocommendations_For_Index,count_row)

Coca Cola - Songs.pk - 128Kbps.mp3
DJ Snake feat. Justin Bieber - Let Me Love You.mp3
Bebe Rexha - Drinking About You (The Chainsmokers Remix).mp3
